# Impact of GDP per capita on employment in worldwide agriculture over the past 30 years  

Imports and set magics:

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib_venn import venn2
from functools import reduce

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

# import the API for WorldBank-data
from pandas_datareader import wb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read data

Here we are importing data using the API for WorldBank-data. We found the following data and data-codes using the Website of the WorldBank to provide us with different measures for countrys worldwide over the past decades:\
#
-Arable land in hectares --> *AG.LND.ARBL.HA*\
-Labor force --> *SL.TLF.TOTL.IN*\
-GDP per capita in USD --> *NY.GDP.PCAP.CD*\
-Employment in agriculture in % of total empoyment --> *SL.AGR.EMPL.ZS*

In [21]:
# Setting up list of ISO-codes of countries where we are going to import data from using the API
countries_ISO_list = ['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM', 'AUS', 'AUT', 
                      'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 
                      'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 
                      'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 
                      'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 
                      'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FRO', 'GAB', 'GBR', 
                      'GEO', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 
                      'GUY', 'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN', 
                      'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 
                      'KIR', 'KNA', 'KOR', 'KWT', 'LBN', 'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 
                      'LTU', 'LUX', 'LVA', 'MAC', 'MAF', 'MAR', 'MCO', 'MDA', 'MDG', 'MDV', 'MEX', 
                      'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 
                      'NAM', 'NCL', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NRU', 'NZL', 'OMN', 
                      'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'POL', 'PRI', 'PRK', 'PRT', 'PRY', 
                      'PSE', 'PYF', 'QAT', 'ROU', 'RUS', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLE', 
                      'SLV', 'SMR', 'SOM', 'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 
                      'SXM', 'SYC', 'SYR', 'TCA', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON', 
                      'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UGA', 'UKR', 'URY', 'USA', 'UZB', 'VCT', 
                      'VEN', 'VGB', 'VIR', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB', 'ZWE']

# Importing the different DataFrames from the WorldBank using the API from the lecture and resetting the index for each DataFrame
wb_arable_land = wb.download(indicator='AG.LND.ARBL.HA', country= countries_ISO_list, start=1989, end=2019)
wb_arable_land = wb_arable_land.reset_index()
wb_labor_force = wb.download(indicator='SL.TLF.TOTL.IN', country= countries_ISO_list, start=1989, end=2019)
wb_labor_force = wb_labor_force.reset_index()
wb_emp_ag = wb.download(indicator='SL.AGR.EMPL.ZS', country= countries_ISO_list, start=1989, end=2019)
wb_emp_ag = wb_emp_ag.reset_index()
wb_gdp_cap = wb.download(indicator='NY.GDP.PCAP.CD', country= countries_ISO_list, start=1989, end=2019)
wb_gdp_cap = wb_gdp_cap.reset_index()

# Resetting the countries as the index for each DataFrame
wb_arable_land = wb_arable_land.set_index('country')
wb_labor_force = wb_labor_force.set_index('country')
wb_emp_ag = wb_emp_ag.set_index('country')
wb_gdp_cap = wb_gdp_cap.set_index('country')

# Clean and merge data 

In [32]:
# changing the data-type of the 'year'-column to an integer
wb_arable_land.year = wb_arable_land.year.astype(int)
wb_labor_force.year = wb_labor_force.year.astype(int)
wb_emp_ag.year = wb_emp_ag.year.astype(int)
wb_gdp_cap.year = wb_gdp_cap.year.astype(int)

# Merging all the DataFrames into one big one
data_frames_list = [wb_arable_land, wb_labor_force, wb_emp_ag, wb_gdp_cap]
data_frames_combined = reduce(lambda left, right: pd.merge(left, right, on = ['country', 'year'], how = 'inner'), data_frames_list)

# Renaming the columns
namedict = {
            'AG.LND.ARBL.HA':'Arable_Land',
            'SL.TLF.TOTL.IN':'Labor_Force',
            'SL.AGR.EMPL.ZS':'Empl_in_agri_in_%_of_all_empl',
            'NY.GDP.PCAP.CD':'GDP_per_capita'}
data_frames_combined.rename(columns=namedict, inplace=True)

# Dropping all rows that contain empty values (NAN)
data_frames_combined.dropna(inplace=True)

# Rounding the floats in columns 'Empl_in_agri_in_%_of_all_empl' and 'GDP_per_capita' 
data_frames_combined['Empl_in_agri_in_%_of_all_empl'] = data_frames_combined['Empl_in_agri_in_%_of_all_empl'].round(2)
data_frames_combined['GDP_per_capita'] = data_frames_combined['GDP_per_capita'].round(2)

# Converting the values in the columns into their respective units

data_frames_combined['Empl_in_agri_in_%_of_all_empl'] = data_frames_combined['Empl_in_agri_in_%_of_all_empl'].apply(lambda x: '{:.2f}%'.format(x*1))

data_frames_combined['GDP_per_capita'] = data_frames_combined['GDP_per_capita'].apply(lambda x: '${:,.2f}'.format(x))

data_frames_combined['Arable_Land'] = data_frames_combined['Arable_Land'].apply(lambda x: '{:,.0f} ha'.format(x))

data_frames_combined.Labor_Force = data_frames_combined.Labor_Force.astype(int)

data_frames_combined.head(50)

,year,Arable_Land,Labor_Force,Empl_in_agri_in_%_of_all_empl,GDP_per_capita
country,,,,,
Afghanistan,2019,"7,788,000 ha",9991682,42.50%,$500.52
Afghanistan,2018,"7,794,000 ha",9613728,43.13%,$502.06
Afghanistan,2017,"7,699,000 ha",9254594,43.99%,$530.15
Afghanistan,2016,"7,729,000 ha",8913938,45.81%,$520.25
Afghanistan,2015,"7,765,000 ha",8630723,47.07%,$592.48
Afghanistan,2014,"7,771,000 ha",8285362,48.41%,$628.15
Afghanistan,2013,"7,785,000 ha",7881567,49.82%,$651.99
Afghanistan,2012,"7,790,000 ha",7520865,51.40%,$663.14
Afghanistan,2011,"7,791,000 ha",7098230,53.56%,$621.91


## Exploring the individual data sets 

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [2]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Explain what you see when moving elements of the interactive plot around. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.